### Import

In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/raw/index_1.csv')
df.head()

,date,datetime,cash_type,card,money,coffee_name
0,2024-03-01,2024-03-01 10:15:50.520,card,ANON-0000-0000-0001,38.7,Latte
1,2024-03-01,2024-03-01 12:19:22.539,card,ANON-0000-0000-0002,38.7,Hot Chocolate
2,2024-03-01,2024-03-01 12:20:18.089,card,ANON-0000-0000-0002,38.7,Hot Chocolate
3,2024-03-01,2024-03-01 13:46:33.006,card,ANON-0000-0000-0003,28.9,Americano
4,2024-03-01,2024-03-01 13:48:14.626,card,ANON-0000-0000-0004,38.7,Latte


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3636 entries, 0 to 3635
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         3636 non-null   object 
 1   datetime     3636 non-null   object 
 2   cash_type    3636 non-null   object 
 3   card         3547 non-null   object 
 4   money        3636 non-null   float64
 5   coffee_name  3636 non-null   object 
dtypes: float64(1), object(5)
memory usage: 170.6+ KB


In [6]:
df.isna().sum()

date            0
datetime        0
cash_type       0
card           89
money           0
coffee_name     0
dtype: int64

In [7]:
df.isna().mean() * 100

date           0.000000
datetime       0.000000
cash_type      0.000000
card           2.447745
money          0.000000
coffee_name    0.000000
dtype: float64

In [8]:
df['coffee_name'].value_counts()

coffee_name
Americano with Milk    824
Latte                  782
Americano              578
Cappuccino             501
Cortado                292
Hot Chocolate          282
Cocoa                  243
Espresso               134
Name: count, dtype: int64

In [9]:
df['cash_type'].value_counts()

cash_type
card    3547
cash      89
Name: count, dtype: int64

In [10]:
df.duplicated(subset=['card'], keep=False).value_counts()

True     2865
False     771
Name: count, dtype: int64

### Clear missing value

In [11]:
df_clean = df.dropna(subset=['card'])

In [12]:
df_clean.isna().sum()

date           0
datetime       0
cash_type      0
card           0
money          0
coffee_name    0
dtype: int64

In [13]:
df_final = df_clean[['datetime', 'cash_type', 'card', 'money', 'coffee_name']]

In [14]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3547 entries, 0 to 3635
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   datetime     3547 non-null   object 
 1   cash_type    3547 non-null   object 
 2   card         3547 non-null   object 
 3   money        3547 non-null   float64
 4   coffee_name  3547 non-null   object 
dtypes: float64(1), object(4)
memory usage: 166.3+ KB


### Explore data

### 1. Waktu vs Penjualan (Forecasting)

Bagaimana tren nilai transaksi dari waktu ke waktu?
Apakah terdapat pola harian, mingguan, atau musiman pada nilai transaksi?
Hari atau jam apa yang menghasilkan total transaksi tertinggi?
Bagaimana estimasi nilai transaksi pada periode berikutnya?

### 2. Produk vs Penjualan

Produk kopi apa yang memberikan kontribusi terbesar terhadap total nilai transaksi?
Produk mana yang penjualannya stabil dan mana yang fluktuatif?
Bagaimana perbedaan pola penjualan tiap produk berdasarkan waktu?

### 3. Metode Pembayaran vs Nilai Transaksi

Apakah terdapat perbedaan rata-rata nilai transaksi antara cash dan cashless?
Metode pembayaran mana yang menghasilkan nilai transaksi lebih besar?
Bagaimana tren penggunaan cash vs cashless dari waktu ke waktu?

### 4. Produk vs Metode Pembayaran

Produk kopi apa yang lebih sering dibayar menggunakan cash atau cashless?
Apakah preferensi metode pembayaran berbeda untuk setiap produk?


### 5. Kombinasi Waktu, Produk, dan Pembayaran

Produk apa yang paling laku pada waktu tertentu?
Metode pembayaran apa yang dominan pada hari atau jam tertentu?
Kombinasi produk dan metode pembayaran apa yang paling berkontribusi terhadap pendapatan?

### 6. Pertanyaan Khusus Forecasting

Berapa estimasi total nilai transaksi pada hari/minggu/bulan berikutnya?
Produk apa yang perlu diprioritaskan stoknya berdasarkan proyeksi penjualan?
Apakah perubahan metode pembayaran memengaruhi proyeksi pendapatan?